In [18]:
from openai import api_key
from pydantic import SecretStr
from langchain_openai import ChatOpenAI

api_key = SecretStr(input("Enter your OpenAI API Key:"))

model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Note: LLMs are not all knowing

They have training cutoff dates, which means they cannot answer questions about events after that date.

The model that we are using (gpt-4o-mini) has a training cutoff of October 2023


In [ ]:
model.invoke("What's on the front page of hacker news?").content

# Enter tool calls

Tool calls allow us to augment the LLM with current information, provide it tools to do meaningful work, or anything else that can be expressed with code

In [35]:
from langchain_core.tools import tool
import requests

@tool
def read_webpage(url: str) -> str:
    """
    Fetch a url's content and return it as a string
    """
    print("Fetching content from", url)
    response = requests.get(url)
    
    return response.text


tools = [read_webpage]

In [36]:
tool_model = model.bind_tools(tools)

In [ ]:
from langchain_core.messages import AIMessage
from typing import cast

call: AIMessage = cast(AIMessage, tool_model.invoke("what's on the front page of hacker news?"))

print(repr(call.content))
print(call.tool_calls)

# State Machines - Can be thought of as a directed graph

* States (Nodes)
	* The current _state_ of the graph. Dictates what action should be taken
* Transitions (Edges)
	* The list of possible next values for state. Not all transitions are possible.
		* Eg: A traffic light has 3 transitions. Green => Yellow, Yellow => Red and Red => Green. Yellow => Green should be impossible.

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

system_prompt = """
    You must provide your answers in a format readable in a terminal. Do not provide any text other than the answer.
    Your responses must not exceed 120 characters in width but can be multiple lines.
"""


def call_agent(state: MessagesState):
    response = tool_model.invoke([
        ("system", system_prompt),
        *state["messages"]
    ])

    return {
        "messages": [response]
    }

tool_node = ToolNode(tools)

def after_prompt_edge(state: MessagesState):
    if cast(AIMessage, state["messages"][-1]).tool_calls:
        return tool_node.name
    return END


workflow = StateGraph(MessagesState)

workflow.add_node(call_agent)

workflow.add_node(tool_node)

workflow.add_edge(START, call_agent.__name__)
workflow.add_edge(tool_node.name, call_agent.__name__)
workflow.add_conditional_edges(call_agent.__name__, after_prompt_edge)

app = workflow.compile()

app.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
       +------------+         
       | call_agent |         
       +------------+         
         .         .          
       ..           ..        
      .               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


# Agentic / Tool Calling Demo

Let's try that first example again, this time with our defined state machine

In [49]:
messages = [
    HumanMessage(content="Return the top 10 stories on hacker news, their titles, URLs, and comment URLs.")
]

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print(final_state['messages'][-1].content)

Fetching content from https://news.ycombinator.com/
1. [Debugging: Indispensable rules for finding even the most elusive problems (2004)](https://dwheeler.com/essays/debugging-agans.html)  
   Comments: [135](https://news.ycombinator.com/item?id=42682602)  

2. [A Laptop Stand Made from a Single Sheet of Recycled Paper](https://www.core77.com/posts/134948/A-Laptop-Stand-Made-from-a-Single-Sheet-of-Recycled-Paper)  
   Comments: [104](https://news.ycombinator.com/item?id=42662329)  

3. [I created an open-source Hardware Hacking Wiki – with tutorials for beginners](https://www.hardbreak.wiki)  
   Comments: [16](https://news.ycombinator.com/item?id=42672821)  

4. [Subway Stories](https://subwaystories.nyc/)  
   Comments: [15](https://news.ycombinator.com/item?id=42684211)  

5. [Lines of code that will beat A/B testing every time (2012)](https://stevehanov.ca/blog/index.php?id=132)  
   Comments: [14](https://news.ycombinator.com/item?id=42650954)  

6. [The Origins of Wokeness](https

We can even ask follow up questions!

In [ ]:

question = input("What is your follow-up question?")
messages.append(HumanMessage(content=question))

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print("You asked:", question)
print(final_state['messages'][-1].content)

# Final demo

Because the tool that we gave our agent was general, not specific, we are not limited to fetching results from hacker news!

In [ ]:
state = app.invoke({"messages": [
	HumanMessage(content="What are the recent blog posts posted by Source Allies?"),
]})

print(state['messages'][-1].content)